# TASK 1 : Feature Engineering Submission - Arin Dhawan

Aim: To come up with 5 new features apart from the existing and in-development ones. 

### FEAT 1 : Trade Momentum

In [ ]:
def calculate_trade_momentum(trades):
    returns = trades['Close'].pct_change().dropna()
    momentum = returns.rolling(window=10).sum()
    trades['Trade_Momentum'] = momentum
    return trades

### FEAT 2: Order Cancelation Ration

In [ ]:
def calculate_order_cancellation_ratio(trades):
    trades['Order_Cancellation_Ratio'] = trades['Cancelled_Orders'] / trades['Total_Orders']
    return trades


### FEAT 3 : VWAP Deviation

In [ ]:
def calculate_vwap_deviation(trades):
    trades['VWAP'] = (trades['Volume'] * trades['Close']).cumsum() / trades['Volume'].cumsum()
    trades['VWAP_Deviation'] = (trades['Close'] - trades['VWAP']) / trades['VWAP']
    return trades


### FEAT 4 : Order Flow Imbalance

In [ ]:
def calculate_order_flow_imbalance(trades):
    trades['Order_Flow_Imbalance'] = (trades['Buy_Orders'] - trades['Sell_Orders']) / (trades['Buy_Orders'] + trades['Sell_Orders'])
    return trades


### FEAT 5 : Price Reversion

In [ ]:
def calculate_price_reversion_indicator(trades):
    trades['Price_Reversion_Indicator'] = trades['Close'] - trades['Close'].rolling(window=20).mean()
    return trades
